In [31]:
import math
import random

class V:
    def __init__(self, value, op = None , parents = None, label = None):
        self.value = value
        self.grad = 0
        self.parents = parents
        self.label = label
        self.op = op
        self._backward = lambda: None

    def __add__(self, other):
        other = V._cast_to_v(other)
        new_value = self.value + other.value
        result = V(new_value, op='+', parents=(self,other))

        def backward():
            self.grad += result.grad * 1
            other.grad += result.grad * 1
                            
        result._backward = backward

        return result

    def __mul__(self, other):
        other = V._cast_to_v(other)
        new_value = self.value * other.value
        result = V(new_value, op='*', parents=(self,other))

        def backward():
            self.grad += result.grad * other.value
            other.grad += result.grad * self.value

        result._backward = backward
        
        return result  
    
    def __pow__(self, other):
        assert isinstance(other, (float, int)), "only float and int supported in pow"

        new_value = self.value ** other
        result = V(new_value, op='**', parents=(self,))

        def backward():
            self.grad += other * (self.value ** (other - 1)) * result.grad 

        result._backward = backward

        return result

    def __radd__(self, other):
        return self.__add__(other)

    def __sub__(self, other):
        other = V._cast_to_v(other)
        return self.__add__(other.__mul__(-1))
    
    def __rsub__(self, other):
        return self.__sub__(other)
    
    def __rmul__(self,other):
        return self.__mul__(other)
    
    def tanh(self):
        new_value = tanh(self.value)
        result = V(new_value, op='tanh', parents=(self,))

        def backward():
            self.grad += result.grad * (1 - new_value ** 2)

        result._backward = backward

        return result

    def __repr__(self) -> str:
       return f"{self.label}: value={self.value} grad={self.grad} op={self.op}"

    def backward(self, isFirst = True):
        if isFirst:
            self.grad = 1.0   
            self._backward()
        
        if self.parents == None:
            return

        for parent in self.parents:
            parent._backward()    

        for parent in self.parents:
            parent.backward(False)
 
    @staticmethod
    def _cast_to_v(other):
        return other if isinstance(other, V) else V(other)
    
def tanh(value):
    e2x = math.exp(2 * value) 
    top = e2x - 1
    bottom = e2x + 1
    new_value = top / bottom
    return new_value


In [32]:
assert V(1).value == 1 
v3 = V(1) + V(2)
assert v3.value == 3
v4 = V(2) * V(8)
assert v4.value == 16
tanh(0.1)

0.09966799462495583

f = a * x

df/dx = [(a * (x+h)) - a * x] / h
df/dx = [ax + ah - ax] / h
df/dx = [ah]/h = a


g = b + y
dg/dy =  [b + y + h - (b +y)] /h
dg/dy =  [b + y + h - b -y] /h
dg/dy =  1

L = f + g
dL/df = 1
dL/dg = 1

g = b + y
dL/db = 1 * dL/df(1)
dL/dy = 1 * dL/df(1)

f = a * x
dL/da = x * dL/dg(1)  
dL/dx = a * dL/dg(1) 

for a = 3 x = 4 b = 6 y = 7
L = f + g
dL/df = 1
dL/dg = 1

g = b + y
dL/db = 1 * dL/df(1)  = 1
dL/dy = 1 * dL/df(1)  = 1

f = a * x
dL/da = x * dL/dg(1) = 4
dL/dx = a * dL/dg(1) = 3

In [33]:
a = V(3, label='a')
x = V(4, label='x')
b = V(-4.5, label='b')
y = V(-7, label='y')


f = a * x; f.label = 'f'
g = b + y; g.label = 'g'
L = f + g; L.label = 'L'
tan_L  = L.tanh(); tan_L.label = 'tanh(L)'

tan_L.backward()






In [34]:
from typing import Any


class Neuron():
    def __init__(self, weights_count):
        self.weights = [V(random.uniform(-1,1)) for _ in range(weights_count)]
        self.bias = V(random.uniform(-1,1))

    def __call__(self, inputs):
        assert len(inputs) == len(self.weights), "Inputs should have same length as weights len"
        result = sum([i * w for (i,w) in zip(inputs,self.weights)]) + self.bias
        return result.tanh()

    def params(self):
        return [self.bias] + self.weights 

class Layer():       
    def __init__(self, neurons_count, weights_count_per_neuron):
        self.neurons = [Neuron(weights_count_per_neuron) for _ in range(neurons_count)]

    def __call__(self, inputs):
        return [neuron(inputs) for neuron in self.neurons]

    def params(self):
        return [params for neuron in self.neurons for params in neuron.params()]

class Net():
    def __init__(self, layer_sizes):
        self.layers = [Layer(layer_sizes[index+1],layer_sizes[index]) for index in  range(len(layer_sizes) - 1)]
    
    def __call__(self, inputs):
        for layer in self.layers:
            inputs = layer(inputs)
        return unwrap_if_single_element(inputs) 
    
    def params(self):
        return [params for layer in self.layers for params in layer.params()]

def unwrap_if_single_element(array):
    return array[0] if len(array) == 1 else array


In [37]:
net = Net([3,4,1])

inputs = [
    [1,2,3],
    [5,4,1],
    [3,2,5]
]
expected_outputs =     [1,0,1]


def loss(actual_outputs, expected_outputs):
    return sum([(actual - expected) ** 2 for actual, expected in zip(actual_outputs, expected_outputs)])

def forward(net, inputs):
    actual_outputs = []
    for single_input in inputs:
        actual_outputs.append(net(single_input))
    return actual_outputs

def step(net, inputs, step_size = 0.001):
    for param in net.params():
        param.grad = 0.0
    actual_outputs = forward(net, inputs)
    cost = loss(actual_outputs, expected_outputs)
    cost.backward()

    for param in net.params():
        param.value -= step_size * param.grad

    return cost

for i in range(10000):
    cost = step(net, inputs)

for input in inputs:
    print(net(input))





None: value=0.9739032821337767 grad=0 op=tanh
None: value=-0.00024356531175706204 grad=0 op=tanh
None: value=0.9785549003781723 grad=0 op=tanh
